# Using Tools
This notebook demonstrates how we can use tools to gather information and provide more accurate responses. We'll build a weather-checking mock as an example.

## What we'll learn:
- Basic interaction with Language Models (LLM)
- How to create and use tools with AI
- The complete flow of an AI agent using tools
- Understanding the message flow in a tool-enabled conversation

### Setup

In [1]:
import json
from dotenv import load_dotenv
from lib.messages import UserMessage, SystemMessage, ToolMessage  # Different message types
from lib.tooling import tool  # Tool decorator for creating AI tools
from lib.llm import LLM  # Our Language Model wrapper

In [2]:
load_dotenv()

False

In [3]:
chat_model = LLM()

## Basic LLM Interaction
Before we dive into tools, let's understand how to interact with our Language Model in its simplest form. 
There are two main ways to communicate with the model:

In [4]:
# Method 1: Simple single-turn query
response = chat_model.invoke("What is an AI Agent?")
print("Single Query Response:\n", response.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: voc-1454**************************************6929. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
# Method 2: Multi-turn conversation with specific roles
messages = [
    SystemMessage(content="You're an OpenAI API specialist"),
    UserMessage(content="What is Function Calling?")
]
response = chat_model.invoke(messages)
print("\nStructured Conversation Response:\n", response.content)

## Building an AI Tool
Now let's make our AI more capable by giving it a tool to check the weather. 
This demonstrates how we can extend AI capabilities beyond just conversation.

### Understanding the Tool Structure:
1. We use the `@tool` decorator to mark a function as an AI tool
2. The tool needs clear documentation and typed parameters
3. The tool should return structured data

In [ ]:
@tool
def get_weather(city: str):
    """Get the current temperature for a city.
    
    Args:
        city (str): Name of the city to check weather for
        
    Returns:
        dict: Contains temperature information for the requested city
    """
    # In a real application, this would call a weather API
    mock_weather = {
        "São Paulo": "28°C",
        "Oslo": "-3°C",
        "New York": "15°C",
        "Tokyo": "22°C"
    }
    return {"temperature": mock_weather.get(city, "Unknown")}

In [ ]:
# Bind the tool to an LLM
chat_model_with_tools = LLM(tools=[get_weather])

## Understanding the Tool Usage Flow
Let's break down how the AI uses tools step by step:

1. User asks a question about weather
2. AI recognizes the need to use the weather tool
3. AI makes a tool call
4. Tool executes and returns results
5. AI processes the tool's response
6. AI formulates a natural language response

Let's see this in action:

In [ ]:
# Set up our system with clear instructions
messages = [
    SystemMessage(
        content="You are a helpful assistant that can access a tool to get current temperature " 
                "for cities. Use the tool whenever someone asks about the weather or temperature " 
                "in a specific location. Infor the user if you don't know the answer."
    ),
    UserMessage(content="How cold is it in Oslo?")
]

In [ ]:
# AI decides to use the weather tool
ai_message = chat_model_with_tools.invoke(messages)
ai_message

In [ ]:
# Check messages structure
messages.append(ai_message)
messages

In [ ]:
# Tool call id will be required later when creating the ToolMessage
tool_call_id = messages[-1].tool_calls[0].id
tool_call_id

In [ ]:
# Extract the arguments
args = json.loads(messages[-1].tool_calls[0].function.arguments)
args

In [ ]:
# Execute the tool with the AI's requested parameters
tool_result = get_weather(**args)
tool_result

In [ ]:
# Create a tool response message
tool_message = ToolMessage(
    content=tool_result["temperature"], 
    tool_call_id=tool_call_id, 
    name="get_weather"
)
tool_message

In [ ]:
# Check messages structure
messages.append(tool_message)
messages

In [ ]:
# Let AI formulate final response
ai_message = chat_model_with_tools.invoke(messages)
ai_message

In [ ]:
# Check messages structure
messages.append(ai_message)
messages